In [3]:
from utils import (
    get_cross_validation_results,
    get_final_model_performance,
    TransactionDataset,
    TuneHyperParams,
)

from constants import Columns, ModelConstants, ResamplingStrategy

In [4]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NearMiss, TomekLinks, ClusterCentroids


In [5]:
data = TransactionDataset().get_training_test_split()

In [6]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1']]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [7]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [8]:
cluster_sampling = ClusterCentroids(random_state=ModelConstants.RANDOM_STATE)
tomek_sampling = TomekLinks()
near_miss_sampling = NearMiss()

In [9]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("sampling", cluster_sampling),
        ("knn", knn_classifier)
    ]
)

In [10]:
tuning_params = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
        "passthrough"
    ],

    "sampling": [
        cluster_sampling,
        tomek_sampling,
        near_miss_sampling,
    ],
    "sampling__sampling_strategy": [
        ResamplingStrategy.MINORITY_ONLY,
        ResamplingStrategy.ALL,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
    ],

    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21, 25],
    "knn__weights": ["uniform", "distance"],
}

In [11]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

/Users/mladenrm/.local/share/virtualenvs/ML_Challenge-_INNdbtJ/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mladenrm/.local/share/virtualenvs/ML_Challenge-_INNdbtJ/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mladenrm/.local/share/virtualenvs/ML_Challenge-_INNdbtJ/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mladenrm/.local/share/virtualenvs/ML_Challenge-_INNdbtJ/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The de

{'sampling__sampling_strategy': 0.2, 'sampling': NearMiss(sampling_strategy=0.2), 'knn__weights': 'uniform', 'knn__n_neighbors': 11, 'column_transformation': ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Customer_Type']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['0', '0.2', '0.4',
                                                             '0.6', '0.8',
                                                             '1']]),
                                 ['SpecificHoliday']),
                                ('powertransformer', PowerTransformer(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1390c1e10>)])}
Best parameter (CV score: 0.599):


In [12]:
get_cross_validation_results(best_model, data.TRAINING.predictors, data.TRAINING.outcome)

,Metric for Training Set,Score
0,fit_timemean score,0.243255
1,score_timemean score,0.150988
2,5-fold CV F1 Scoremean score,0.599059
3,5-fold CV Accuracymean score,0.890107
4,5-fold CV Balanced Accuracymean score,0.742676
5,5-fold CV Matthew's Correlation Coefficientmea...,0.544627


In [13]:
get_final_model_performance(best_model, data.TRAINING, data.TESTING)

,Metric for Testing Set,Score
0,F1 Score,0.615385
1,Accuracy,0.895535
2,Balanced Accuracy,0.821786
3,Matthew's Correlation Coefficient,0.565349
